In [1]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 48.0 MB/s eta 0:00:00:00:0100:01


In [2]:
!pip install -U sentence-transformers

# 1. 데이터 로드

In [3]:
import numpy as np
import os
import pandas as pd
import urllib.request
import faiss
import time
from sentence_transformers import SentenceTransformer

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/19.%20Topic%20Modeling%20(LDA%2C%20BERT-Based)/dataset/abcnews-date-text.csv", filename="abcnews-date-text.csv")

df = pd.read_csv("abcnews-date-text.csv")
data = df.headline_text.to_list()

In [11]:
data = data[:10000]

In [12]:
len(data)

10000

In [5]:
# 상위 5개의 샘플 출력
data[:5]

['aba decides against community broadcasting licence',
 'act fire witnesses must be aware of defamation',
 'a g calls for infrastructure protection summit',
 'air nz staff in aust strike for pay rise',
 'air nz strike to affect australian travellers']

In [6]:
print('총 샘플의 개수 :', len(data))

총 샘플의 개수 : 1082168


In [13]:
df.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


# 2. SBERT 임베딩

In [14]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(data)
print('임베딩 된 벡터 수 :', len(encoded_data))

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

임베딩 된 벡터 수 : 10000


# 3. 인덱스 정의 및 데이터 추가

In [15]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(data))))

faiss.write_index(index, 'abc_news')

# 4. 검색 및 시간 측정

In [16]:
def search(query):
   t = time.time()
   query_vector = model.encode([query])
   k = 5
   top_k = index.search(query_vector, k)
   print('total time: {}'.format(time.time() - t))
   return [data[_id] for _id in top_k[1].tolist()[0]]


In [17]:
query = str(input())
results = search(query)

print('results :')
for result in results:
   print('\t', result)

 Underwater Forest Discovered


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

total time: 0.05488872528076172
results :
	 portland centre moves closer to underwater display
	 scud powers through in miami
	 moya moves into miami quarters
	 boy drowns on hinterland property
	 tourist drowns on reef


In [18]:
input()

 underwater


'underwater'

In [21]:
data[0]

'aba decides against community broadcasting licence'

In [23]:
data[5000]

'sri lanka hoping for new zealand defeat'